## tradeSeq for DE along Wolffian rostro-caudal axis in male reproductive tract

In [ ]:
#if (!require("BiocManager", quietly = TRUE))
#    install.packages("BiocManager")

#BiocManager::install("tradeSeq")

In [ ]:
#install.packages('igraph')

In [ ]:
ip = as.data.frame(installed.packages()[,c(1,3:4)])
ip = ip[is.na(ip$Priority),1:2,drop=FALSE]
head(ip)

In [ ]:
library(tradeSeq)
library(SingleCellExperiment)
library(slingshot)

In [ ]:
adata <- readRDS("/lustre/scratch126/cellgen/team292/vl6/VISIUM/malereproductiveaxis_epi_cytassist_downsampled.rds")
adata


In [ ]:
counts <- assay(adata, "X")

In [ ]:
set.seed(7)
pseudotime <- colData(adata)$MaleReproductiveAxis
cellWeights <- rep(1, 114)

In [ ]:
#icMat <- evaluateK(counts = counts, sds = SlingshotDataSet(adata), k = 3:10, 
#                   nGenes = 200, verbose = T)

In [ ]:
#BiocManager::install("scran")

In [ ]:
library(scran)

In [ ]:
libsizes <- colSums(counts)
size.factors <- libsizes/mean(libsizes)
logcounts(adata) <- log2(t(t(counts)/size.factors) + 1)
assayNames(adata)

In [ ]:
# Filter genes with non-zero biological components
getVariableGenes <- function(adata,fdr_thresh=NULL,var_thresh=0) {
  gene_var <- modelGeneVar(adata)
  genes <- getTopHVGs(gene_var, var.threshold =var_thresh ,fdr.threshold = fdr_thresh)
  return(genes)
}

In [ ]:
genes <- getVariableGenes(adata)

In [ ]:
length(genes)

In [ ]:
dim(counts[genes,])

In [ ]:
sce <- fitGAM(counts = counts[genes,], pseudotime = pseudotime, cellWeights = cellWeights, nknots = 6,
                verbose = TRUE, parallel = TRUE)

In [ ]:
table(rowData(sce)$tradeSeq$converged)

In [ ]:
adata <- adata[genes, ]

In [ ]:
adata

In [ ]:
gamList <- fitGAM(counts = counts[genes,], pseudotime = pseudotime, cellWeights = cellWeights, nknots = 6,
                verbose = TRUE, 
                  parallel = TRUE, sce = FALSE)

In [ ]:
summary(gamList[["SRD5A2"]])


In [ ]:
pvalLineage <- getSmootherPvalues(gamList)
statLineage <- getSmootherTestStats(gamList)

### ASSOCIATION TEST

A first exploration of the data analysis may consist of checking whether gene expression is associated with a particular lineage. The statistical test performed here, implemented in the associationTest function, is testing the null hypothesis that all smoother coefficients are equal to each other. This can be interpreted as testing whether the average gene expression is significantly changing along pseudotime.



In [ ]:
assoRes <- associationTest(sce)
head(assoRes)

In [ ]:
write.csv(assoRes, file = '/nfs/team292/vl6/FetalReproductiveTract/manuscript_v1/supp_tables/SupplementaryTable8_wolffian_epi_cytassist.csv')


### FILTER GENES AND CLUSTER THEM BY PATTERN

In [ ]:
dim(assoRes)

In [ ]:
assoRes = assoRes[assoRes$pvalue < 0.001,]
dim(assoRes)

In [ ]:
assoRes = assoRes[assoRes$meanLogFC > 0.5,]
dim(assoRes)

In [ ]:
assoRes['ESR1',]

In [ ]:
assoRes = assoRes[order(assoRes$waldStat, decreasing = TRUE),]

In [ ]:
head(assoRes)

In [ ]:
# Get the index of the DataFrame
tops <- row.names(assoRes)[1:200]

In [ ]:
tops

In [ ]:
library(ggplot2)

In [ ]:
for (gene in tops) {
    print(gene)
    print(plotSmoothers(sce, counts, gene = gene, alpha = 0) + ggtitle(gene) +
                          theme(title = element_text(size = 20, face = 'bold'),
                                axis.title = element_text(size = 20),
                                axis.text = element_text(size = 20),
                                legend.text = element_text(size = 20),
                                legend.title = element_text(size = 20, face = "bold")))
}

In [ ]:
sce

In [ ]:
model_vals_df <- predictSmooth(models = sce, gene=row.names(assoRes), nPoints=100) # output is 715 genes x 100 pseudospace values 

In [ ]:
model_vals_mtx <- predictSmooth(models = sce, gene=row.names(assoRes), nPoints=100, tidy=FALSE)

In [ ]:
dim(model_vals_mtx) # 715 genes x 100 pseudospace values 

In [ ]:
model_vals_mtx[1:10, 1:20]

In [ ]:
write.csv(model_vals_mtx, file = '/lustre/scratch126/cellgen/team292/vl6/VISIUM/malereproductiveaxis_epi_cytassist_downsampled_fitted_values_tradeseq.csv')
